# Size evaluation of Value-at-Risk Validation tests

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import vvar as vv

## Specify the params of the Size evaluation

In [2]:
#PLOT_PATH = 'C:\\Users\\dkasz\\OneDrive\\Pulpit\\!_GIT\\VVaR\\plots\\'
BACKTEST_DICT = vv.get_dict_tests() # Syntax
params = {}
T      = [250, 500, 1000]  # Loop over all lengths of observation 
P      = [0.01, 0.05]      # Loop over all VaR levels 
N      = 10000             # Number of simulations per instance

# Predefine output of evalutation
s_results = pd.DataFrame(None, columns=('test_name','t', 'p', 'lr', 'p-value'))

## Loop over T $\times$ P

In [3]:
for t in T: 
    for p in P:
        for _ in range(N):
            # Generate instance of simulation (t x p)
            y   = pd.DataFrame(np.random.rand(t))
            y_p = pd.DataFrame(np.repeat(p, t))

            # Interate over all backtests
            for t_name, f_backtest in BACKTEST_DICT.items():
                #if t_name != 'Kupiec-POF':
                #    continue
                # Create label of the test
                sim_index = '{}_{}_{}'.format(t_name, t, p)
                # Perform the test 
                res = pd.DataFrame(
                            dict(**{'test_name': t_name, 't': t, 'p': p}, 
                                 **f_backtest(y=y, y_p=y_p, p=p, params=params)), 
                            index=[sim_index])
                # Append results
                s_results  = s_results.append(res, ignore_index=False,
                                           sort=False)
            
s_results.sort_index(inplace=True)

D:\Anaconda\envs\Python\lib\site-packages\numpy\lib\function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
D:\Anaconda\envs\Python\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
D:\Anaconda\envs\Python\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
D:\Anaconda\envs\Python\lib\site-packages\scipy\stats\_distn_infrastructure.py:1827: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= np.asarray(_b)) & cond0
C:\Users\dkasz\OneDrive\Pulpit\!_GIT\VVaR\vvar\__init__.py:38: RuntimeWarning: divide by zero encountered in power
  return max((1 - a) ** np.power(d_i - 1, b) - (1 - a) ** np.power(d_i, b), 10**-11)
D:\Anaconda\envs\Python\lib\site-packages\numpy\lib\function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c 

0.05
0.05
0.05


## Plot the *Size* figures

In [4]:
def size_plot(s_results, test, P, T, plot_lim = (0, 1), if_save = False):
    index = np.linspace(0, 1, num=10000)
    line_type = ['-', '--', ':']

    fig = plt.figure(num=None, figsize=(10, 4.24), dpi=80, facecolor='w', edgecolor='k')
    fig.subplots_adjust(wspace=0.3, hspace=0.3, left=0.01, right=0.99, top=0.99, bottom=0.01)

    for i1, p in enumerate(P):
        ax = plt.subplot(1, 2, i1+1) 

        for i2, t in enumerate(T):
            # Select appropriate backtesting results
            p_val = s_results.loc[
                            (s_results['test_name'] == test)
                          & (s_results['p'] == p)
                          & (s_results['t'] == t)
                                 ]['p-value'].dropna().values
            
            p_val_c = np.array([sum((np.array(p_val) <= i) * 1) / len(p_val)
                          for i in index])
            _, ind = np.unique(p_val_c, return_index=True)
            ind = np.sort(ind.tolist() + np.array(ind[1:] - 1).tolist())
            ax.plot(index[ind], p_val_c[ind], label = '$X_1$', color = 'black', linestyle = line_type[i2], linewidth=0.7)

        ax.plot([0, 1], [0, 1], color="red", linewidth=0.5)
        ax.set_xlabel('$H_0$ rejection threshold')
        ax.set_ylabel('Rejection probability')
        ax.set_aspect('equal', 'datalim')
        ax.set_xlim(plot_lim)
        ax.set_ylim(plot_lim)

    if if_save:
        plt.savefig(PLOT_PATH+'{}.pdf'.format(test), bbox_inches='tight')
        plt.close(fig)
    else:
        plt.show()

In [35]:
# Plot results
for test in list(BACKTEST_DICT.keys()):
    size_plot(s_results, test, P, T, plot_lim=(0, 1), if_save=True)

## Table with size evaluation summary

In [13]:
pd.options.display.float_format = '{:,.2f}'.format
size_table = []
for test in list(BACKTEST_DICT.keys()):
    for p in P:
        for t in T:
            p_val = s_results.loc[
                                    (s_results['test_name'] == test)
                                  & (s_results['p'] == p)
                                  & (s_results['t'] == t)
                                         ]['p-value'].dropna().values

            p_val = np.sort(p_val)
            
            p_val = np.array([sum((np.array(p_val) <= i) * 1) / len(p_val)
                          for i in index])
            
            index = np.linspace(0, 1, num=p_val.shape[0])

            ind_O = p_val > index
            ind_U = p_val < index

            T_O = sum(ind_O) / len(index)
            T_U = sum(ind_U) / len(index)

            if sum(p_val[ind_O]) == 0:
                A_O = 0 
            else:
                A_O = np.mean(p_val[ind_O] - index[ind_O])
            if sum(p_val[ind_U]) == 0:
                A_U = 0
            else:
                A_U = np.mean(index[ind_U] - p_val[ind_U])


            A = T_O * A_O + T_U * A_U

            size_table.append({'test':test, 'p':p, 't':t, 'T_O':T_O, 'T_U':T_U, 'A_O':A_O, 'A_U':A_U, 'A':A})

size_table = pd.DataFrame(size_table)

,test,p,t,T_O,T_U,A_O,A_U,A
0,Kupiec-POF,0.01,250,0.56,0.44,0.08,0.09,0.08
1,Kupiec-POF,0.01,500,0.56,0.44,0.05,0.06,0.06
2,Kupiec-POF,0.01,1000,0.42,0.58,0.04,0.04,0.04
3,Kupiec-POF,0.05,250,0.62,0.38,0.04,0.03,0.03
4,Kupiec-POF,0.05,500,0.46,0.54,0.03,0.02,0.02
...,...,...,...,...,...,...,...,...
61,Kramer-GINI,0.01,500,1.00,0.00,0.32,0.00,0.32
62,Kramer-GINI,0.01,1000,1.00,0.00,0.30,0.00,0.30
63,Kramer-GINI,0.05,250,0.99,0.01,0.10,0.00,0.10
64,Kramer-GINI,0.05,500,0.99,0.00,0.09,0.00,0.09
